In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
a=pd.read_csv('Fraud.csv')

In [3]:
a.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score,roc_curve,classification_report,confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

EXPLORING THE DATASET

In [6]:
a.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97225 entries, 0 to 97224
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   step            97225 non-null  int64  
 1   type            97225 non-null  object 
 2   amount          97225 non-null  float64
 3   nameOrig        97225 non-null  object 
 4   oldbalanceOrg   97225 non-null  float64
 5   newbalanceOrig  97225 non-null  float64
 6   nameDest        97225 non-null  object 
 7   oldbalanceDest  97225 non-null  float64
 8   newbalanceDest  97224 non-null  float64
 9   isFraud         97224 non-null  float64
 10  isFlaggedFraud  97224 non-null  float64
dtypes: float64(7), int64(1), object(3)
memory usage: 8.2+ MB


In [7]:
a.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,97225.000000,9.722500e+04,9.722500e+04,9.722500e+04,9.722500e+04,9.722400e+04,97224.000000,97224.0
mean,8.456817,1.724217e+05,8.793150e+05,8.956148e+05,8.792683e+05,1.182315e+06,0.001173,0.0
std,1.833480,3.419651e+05,2.689865e+06,2.727826e+06,2.403354e+06,2.802840e+06,0.034223,0.0
min,1.000000,3.200000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0
25%,8.000000,9.893120e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0
50%,9.000000,5.208135e+04,1.994700e+04,0.000000e+00,2.080800e+04,4.894480e+04,0.000000,0.0
75%,10.000000,2.103607e+05,1.863453e+05,2.107046e+05,5.853365e+05,1.051531e+06,0.000000,0.0
max,10.000000,1.000000e+07,3.379739e+07,3.400874e+07,3.400874e+07,3.397234e+07,1.000000,0.0


In [10]:
a.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [12]:
a.drop(['step'],axis=1)

,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
97220,PAYMENT,17011.46,C1283088834,0.0,0.00,M485693121,0.0,0.0,0.0,0.0
97221,PAYMENT,1680.50,C671281,0.0,0.00,M661110294,0.0,0.0,0.0,0.0
97222,PAYMENT,26450.83,C948744009,0.0,0.00,M1020130183,0.0,0.0,0.0,0.0
97223,PAYMENT,12171.47,C1843902873,0.0,0.00,M2011839805,0.0,0.0,0.0,0.0


In [13]:
a['type'].value_counts()

PAYMENT     38478
CASH_OUT    29839
CASH_IN     19561
TRANSFER     8371
DEBIT         976
Name: type, dtype: int64

In [14]:
b={ "CASH_OUT" :0,
"PAYMENT":1,
"CASH_IN":2,
"TRANSFER":3,
"DEBIT":4,}

In [15]:
a['type']=a['type'].map(b)

In [16]:
a.isna().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    1
isFraud           1
isFlaggedFraud    1
dtype: int64

In [18]:
a=a.dropna()

In [20]:
a.duplicated().sum()

0

In [21]:
a

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,1,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,1,1,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,1,3,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,1,0,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,1,1,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
97219,10,1,10811.91,C504389296,1009.0,0.00,M1318927100,0.0,0.0,0.0,0.0
97220,10,1,17011.46,C1283088834,0.0,0.00,M485693121,0.0,0.0,0.0,0.0
97221,10,1,1680.50,C671281,0.0,0.00,M661110294,0.0,0.0,0.0,0.0
97222,10,1,26450.83,C948744009,0.0,0.00,M1020130183,0.0,0.0,0.0,0.0


In [22]:
scaler=MinMaxScaler()


In [41]:
from sklearn.preprocessing import LabelEncoder
encoder = {}
for i in a.select_dtypes('object').columns:
    encoder[i] = LabelEncoder()
    a[i] = encoder[i].fit_transform(a[i])

In [43]:
x = a.drop(columns=['isFraud'])
y = a['isFraud']

In [44]:
y.value_counts() #class 1 is minority class

0    97110
1      114
Name: isFraud, dtype: int64

In [45]:
over_sample = SMOTE(random_state=0)
x,y = over_sample.fit_resample(x,y)

In [46]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x = scaler.fit_transform(x)


In [47]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.30,random_state=0)

In [48]:
from sklearn.linear_model import LogisticRegression

In [49]:
log_reg = LogisticRegression()
log_reg.fit(x_train,y_train)

LogisticRegression()

In [50]:
y_pred = log_reg.predict(x_test)

In [51]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
confusion_matrix(y_test,y_pred)

array([[25795,  3166],
       [ 3014, 26291]])

In [52]:
accuracy_score(y_test, y_pred)


0.8939347132118216

**ACCURACY TURNS OUT TO BE 89.39%**

In [53]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.89      0.89     28961
           1       0.89      0.90      0.89     29305

    accuracy                           0.89     58266
   macro avg       0.89      0.89      0.89     58266
weighted avg       0.89      0.89      0.89     58266



Trying to improve accuracy by removing multi-collinearity and selecting the best features

In [54]:
from sklearn.model_selection import cross_val_score

In [55]:
cvs = cross_val_score(log_reg,x,y,cv=3)
print(cvs)

[0.62304603 0.90223973 0.93789002]


In [56]:
cvs.mean()

0.82105859334775

CROSS VALIDATION SCORE IS LESSER THAN ACCURACY ; HENCE UNDERFITTING

In [57]:
from sklearn.feature_selection import chi2, SelectKBest

In [59]:
best_fea = SelectKBest(chi2,k=8)
kbest = best_fea.fit_transform(x,y)
np.array(a.drop(columns=['isFraud']).columns)[best_fea.get_support()]

array(['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest'], dtype=object)

In [61]:
pd.DataFrame(list(zip(a.drop(columns=['isFraud']).columns, best_fea.scores_)),columns=['features','scores']).sort_values(by='scores',ascending=False)

,features,scores
0,step,9502.478080
6,nameDest,7531.846066
5,newbalanceOrig,2394.829085
7,oldbalanceDest,1396.307011
2,amount,1296.073190
1,type,793.107955
8,newbalanceDest,634.308907
4,oldbalanceOrg,358.111412
3,nameOrig,92.090447
9,isFlaggedFraud,NaN


 **Now we can build a new logistic regression model using only these 8 features**

In [63]:
x_new = a[['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig','nameDest', 'oldbalanceDest', 'isFlaggedFraud']]
y_new = a['isFraud']

In [64]:
x_new_train, x_new_test, y_new_train, y_new_test = train_test_split(x_new, y_new, test_size=0.30, random_state=0)

In [65]:
log_reg1 = LogisticRegression()
log_reg1.fit(x_new_train,y_new_train)

LogisticRegression()

In [66]:
y2_pred = log_reg1.predict(x_new_test)

In [67]:
confusion_matrix(y_new_test,y2_pred)

array([[29132,     0],
       [   30,     6]])

In [68]:
accuracy_score(y_new_test,y2_pred)

0.9989714755896874

In [69]:
print(classification_report(y_new_test,y2_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     29132
           1       1.00      0.17      0.29        36

    accuracy                           1.00     29168
   macro avg       1.00      0.58      0.64     29168
weighted avg       1.00      1.00      1.00     29168



So final accuracy turns out to be 99.89%, which is a big improvement from previous case

***HENCE NOW THIS MODEL IS SUITABLE FOR BEING USED IN THE PREDICTING ALGORITHM FOR DETECTING FRADULENT TRANSACTIONS***